In [3]:
%load_ext autoreload
%autoreload 2

import duckdb
import os
import pandas as pd
import pathlib

from ibge import *

In [5]:
# Initialize DuckDB connection
conn = duckdb.connect("plants.db")

In [ ]:
conn.execute("SHOW TABLES").df()

,name
0,ibge
1,reflora
2,sibbr


### Arrumar as variações da palavra Brasil na coluna Country